#### 공지사항

API 키(env 파일)가 필요한 경우, 각자 다운 받은 후 push 할 때, 제외시키길 권장함.
보안 문제로 인해 git push가 불가능함

#### 필요한 라이브러리 설치

In [ ]:
# 안될경우 Terminal 창을 VS Code에서 실행해서 직접 하나씩 입력해서 설치 권창창
pip install openai
pip install django-environ
pip install azure-storage-blob

#### gpt-3o-mini(test)+ DALL-E 3 + 통합 코드 이미지 파일 로컬 / Azure Blob Storage 저장

In [ ]:
import os 
import re 
import time
import requests 
import environ
from pathlib import Path
from openai import AzureOpenAI

from azure.storage.blob import BlobServiceClient, BlobClient # Azure Blob Storage 저장을 위한 lib
  
# .env 파일 로드 (모든 환경 변수 로드)
env = environ.Env()
environ.Env.read_env(env_file=".env")  
  
# Azure OpenAI 환경 변수 설정  
AZURE_3OMINI_ENDPOINT = env("AZURE_3OMINI_ENDPOINT")  
AZURE_3OMINI_API_KEY = env("AZURE_3OMINI_API_KEY")  
AZURE_3OMINI_API_VERSION ="2024-12-01-preview"
  
if not AZURE_3OMINI_ENDPOINT or not AZURE_3OMINI_API_KEY:  
    raise ValueError("환경 변수가 올바르게 설정되지 않았습니다. .env 파일을 확인하세요.")  
  

GPT_CLIENT = AzureOpenAI(
    azure_endpoint=AZURE_3OMINI_ENDPOINT,
    api_key=AZURE_3OMINI_API_KEY,
    api_version=AZURE_3OMINI_API_VERSION
)


def generate_prompt_with_gpt3o(user_input):
    try:
        print("GPT-3o-mini를 사용해 프롬프트를 생성합니다...")

        response = GPT_CLIENT.chat.completions.create(
            model="team6-o3-mini",
            messages=[
                {"role": "system", "content": """You are an expert in converting user's natural language descriptions into DALL-E image generation prompts.
            Please generate prompts according to the following guidelines:

            ##Main Guidelines

            1. Carefully analyze the user's description to identify key elements.
            2. Use clear and specific language to write the prompt.
            3. Include details such as the main subject, style, composition, color, and lighting of the image.
            4. Appro-priately utilize artistic references or cultural elements to enrich the prompt.
            5. Add instructions about image quality or resolution if necessary.
            6. Evaluate if the user's request might violate DALL-E's content policy. If there's a possibility of violation, include a message in the user's original language: "This content may be blocked by DALL-E. Please try a different approach." and explain why blocked.
            7. Always provide the prompt in English, regardless of the language used in the user's request.

            ##Prompt Structure

            - Specify the main subject first, then add details.
            - Use adjectives and adverbs effectively to convey the mood and style of the image.
            - Specify the composition or perspective of the image if needed.

            ##Precautions

            - Do not directly mention copyrighted characters or brands.
            - Avoid violent or inappropriate content.
            - Avoid overly complex or ambiguous descriptions, maintain clarity.
            - Avoid words related to violence, adult content, gore, politics, or drugs.
            - Do not use names of real people.
            - Avoid directly mentioning specific body parts.

            ##Using Alternative Expressions

            Consider DALL-E's strict content policy and use visual synonyms with similar meanings to prohibited words. Examples:

            - "shooting star" → "meteor" or "falling star"
            - "exploding" → "bursting" or "expanding"

            ##Example Prompt Format

            "[Style/mood] image of [main subject]. [Detailed description]. [Composition/perspective]. [Color/lighting information]." Follow these guidelines to convert the user's description into a DALL-E-appropriate prompt. The prompt should be creative yet easy for AI to understand. If there's a possibility of content policy violation, notify the user and suggest alternatives."""},
                {"role": "user", "content": user_input}
            ]
        )

        if response.choices and len(response.choices) > 0:
            return response.choices[0].message.content
        else:
            print("응답을 생성하지 못했습니다.")
            return None

    except Exception as e:
        print("GPT-3o-mini 호출 중 예외 발생:", str(e))
        return None
    
# DALL-E 클라이언트 초기화  
DALLE_CLIENT = AzureOpenAI(  
    azure_endpoint=env("AZURE_DALLE_ENDPOINT"),  
    api_key=env("AZURE_DALLE_API_KEY"),  
    api_version=env("AZURE_DALLE_API_VERSION")  
)  

def generate_image_with_dalle(prompt):  
    """DALL-E를 사용해 이미지를 생성"""  
    try:  
        print("DALL-E를 사용해 이미지를 생성합니다...")  
  
        # DALL-E API 호출  
        result = DALLE_CLIENT.images.generate(  
            model="dall-e-3",  # 사용할 DALL-E 모델  
            prompt=prompt,    # 생성할 이미지에 대한 프롬프트  
            n=1               # 생성할 이미지 개수  
        )  
  
        # 결과 처리  
        if result and result.data:  
            image_url = result.data[0].url  
            print("DALL-E 호출 성공! 생성된 이미지 URL:", image_url)  

            # 이미지 다운로드 및 저장
            save_image(image_url, prompt)
            
            return image_url  
        else:  
            print("DALL-E 호출 실패: 결과가 비어 있습니다.")  
            return None  
    except Exception as e:  
        print("DALL-E 호출 중 예외 발생:", str(e))  
        return None  
    
def save_image(image_url, prompt):
    """이미지를 다운로드하여 로컬에 저장"""
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()  

        # 이미지 폴더 생성
        os.makedirs("images", exist_ok=True)

        # 파일명에서 특수문자 제거 및 최대 길이 제한
        sanitized_filename = re.sub(r'[<>:"/\\|?*]', '', prompt[:30]).strip()
        filename = os.path.join("images", f"{sanitized_filename}.png")

        with open(filename, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)

        print(f"이미지가 저장되었습니다: {filename}")

    except Exception as e:
        print(f"이미지 저장 중 오류 발생: {e}")

# 환경 변수 가져오기
account_name = env("STORAGE_ACCOUNT_NAME")  # 스토리지 계정명
account_key = env("STORAGE_ACCOUNT_KEY")  # 스토리지 계정 키
container_name = env("AI_CONTAINER_NAME")  # 컨테이너 이름

# Account Key를 사용하여 BlobServiceClient 생성
blob_service_client = BlobServiceClient(
    f"https://{account_name}.blob.core.windows.net",
    credential=account_key
)

def save_image_to_blob_storage(image_url, prompt):
    """이미지를 다운로드하여 Azure Blob Storage에 저장"""
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()

        # 파일명에서 특수문자 제거 및 최대 길이 제한
        sanitized_filename = re.sub(r'[<>:"/\\|?*]', '', prompt[:30]).strip()
        filename = f"{sanitized_filename}.png"

        # Blob Client 생성
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=filename)

        # 이미지 데이터 업로드 (파일 덮어쓰기 추가)
        blob_client.upload_blob(response.content, overwrite=True)

        # print(f"이미지가 Azure Blob Storage에 저장되었습니다: {filename}")

    except requests.exceptions.RequestException as req_err:
        print(f"이미지 다운로드 오류: {req_err}")

    except Exception as e:
        print(f"Azure Blob Storage에 이미지 저장 중 오류 발생: {e}")

def main():  
    """전체 워크플로 실행"""  
    user_input = input("이미지 생성 아이디어를 입력하세요: ").strip()  
    if not user_input:  
        print("입력값이 비어 있습니다. 유효한 입력값을 제공합니다.")  
        return  
  
    # Step 1: GPT-4o-mini를 사용해 프롬프트 생성  
    prompt = generate_prompt_with_gpt3o(user_input)  
    if not prompt:  
        print("프롬프트 생성 실패. 워크플로를 종료합니다.")  
        return  
  
    # Step 2: DALL-E를 사용해 이미지 생성  
    image_url = generate_image_with_dalle(prompt)  
    if not image_url:  
        print("이미지 생성 실패. 워크플로를 종료합니다.")  
        return  
    
    # Step 3: Azure Blob Storage에 이미지 저장
    if image_url:
        save_image_to_blob_storage(image_url, prompt)
  
    print("\n=== 최종 결과 ===")
    print(f"생성된 이미지가 'images/' 폴더에 저장되었습니다.")   
    print(f"생성된 이미지가 Azure Blob Storage/ai-generated 컨테이너에 저장되었습니다.")  
  
  
if __name__ == "__main__":  
    main()